<a href="https://colab.research.google.com/github/wonjae124/Pytorch-tutorial/blob/main/Transfer_learning_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision.models as models

resnet = models.resnet50(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [2]:
for name, module in resnet.named_children():
    print(name)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [6]:
import torch.nn as nn
num_category = 5
batch_size = 1

class Resnet(nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
        # *를 통해서 unpacking
        # feature extractor : 마지막 fc이전까지의 layer임
        # return 되는 것은 generator 이므로 list로 바꾸어야 함
        self.layer0 = nn.Sequential(*list(resnet.children())[0:-1]) 
        
        # avgpool을 통과하고 나온 텐서를 입력값으로 받음
        self.layer1 = nn.Sequential(
                nn.Linear(2048,500),
                nn.BatchNorm1d(500),
                nn.ReLU(),
                nn.Linear(500,num_category),
                nn.ReLU()
        )
    
    def forward(self,x):
        out = self.layer0(x)
        out = out.view(batch_size,-1)
        out = self.layer1(out)
        return out

In [10]:
model = Resnet()

# layer0 : 학습 OFF
for params in model.layer0.parameters():
    params.require_grad = False

# layer1 : 학습 ON
for params in model.layer1.parameters():
    params.require_grad = True